In [ ]:
# Import libraries
from google.colab import files
import pandas as pd
import re

# Upload the csv file previously downloaded from Spillover Global website
uploaded = files.upload()

# Load the csv file
df = pd.read_csv('SpilloverRankings.csv')

# Print column names
print("Columns in the dataframe:", df.columns)


In [ ]:
# Identify the 'Risk Levels' column
data_column = 'Risk Levels'

# Create extraction function to extract information from each cell in the 'Risk Levels' column
def extract_info(cell):
    pattern = r'Risk Name: (.+?)\nRisk Score: ([0-9.]+)\nImpact Score: ([0-9.]+)\nWeighted Score: ([0-9.]+)\nCorresponding level/levels: ([^\n]*)(?:\n|$)'
    matches = re.findall(pattern, cell, re.MULTILINE)
    return matches

# Apply the extraction function to each row in the 'Risk Levels' column
## Each row represents a virus in the Spillover database
extracted_data = df[data_column].apply(extract_info)

# Print extracted data
print("Extracted data sample:")
print(extracted_data.head())

# Initialize dictionary to store new columns' data
new_columns_data = {}

# Populate the new columns' dictionary with extracted data on the weighted scores
# Ignore other information from the 'Risk Levels' column such as risk_score, impact_score, and corresponding_levels
for idx, entry in extracted_data.items():
    for match in entry:
        risk_name, _, _, weighted_score, _ = match
        risk_name = risk_name.strip()
        # Initialize new column
        if f'{risk_name} - Weighted Score' not in new_columns_data:
            new_columns_data[f'{risk_name} - Weighted Score'] = [None] * len(df)
        # Populate new column with the weighted scores
        new_columns_data[f'{risk_name} - Weighted Score'][idx] = weighted_score.strip()

# Print new columns' data to verify
print("New columns data sample:")
for column_name, column_data in new_columns_data.items():
    print(f"{column_name}: {column_data[:5]}")  # Print first 5 values for each new column

# Create new columns in the dataframe using data from the populated dictionary
for column_name, column_data in new_columns_data.items():
    df[column_name] = column_data

# Confirm columns
print("Final columns in the DataFrame:", df.columns)

# Save the structured data to a new csv file
output_file = 'structured_spillover_ranking_weighted.csv'
df.to_csv(output_file, index=False)

print(f"Data successfully written to {output_file}")

# Download the structured csv file
files.download(output_file)
